# Sample Lya posterior as a function of nuisance parameters

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import numpy as np
import sys
import os
import json
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['savefig.dpi'] = 120
mpl.rcParams['figure.dpi'] = 120
import cProfile
import emcee
import corner
# our own modules
import mf_emulator
import data_MPGADGET
import simplest_emulator
import linear_emulator
import gp_emulator
import data_PD2013
import mean_flux_model
import thermal_model
import pressure_model
import lya_theory
import likelihood
import emcee_sampler

### Setup Lya likelihood function

In [2]:
# read P1D measurement
#data=data_PD2013.P1D_PD2013(blind_data=True,zmin=2.1,zmax=4.7)
data=data_MPGADGET.P1D_MPGADGET(filename="1024_L90_mimic.json",z_list=[2.0,3.0,4.0])
zs=data.z

In [3]:
#basedir='../../p1d_emulator/sim_suites/emulator_512_18062019/'
#basedir='../../p1d_emulator/sim_suites/emulator_512_17052019/'
basedir='../../p1d_emulator/sim_suites/emulator_256_15072019/'
kmax_Mpc=4.0
p1d_label=None
skewers_label='Ns256_wM0.05'
undersample_z=1
max_arxiv_size=2000
paramList=["Delta2_p","n_p","mF","sigT_Mpc","gamma","kF_Mpc"]
emu_type='mf'
if emu_type=='polyfit':
    emu=gp_emulator.GPEmulator(basedir,p1d_label,skewers_label,
                                undersample_z=undersample_z,max_arxiv_size=max_arxiv_size,emu_type=emu_type,
                                kmax_Mpc=kmax_Mpc,verbose=False,paramList=paramList,train=True)
elif emu_type=='k_bin':
    emu=gp_emulator.GPEmulator(basedir,p1d_label,skewers_label,
                               undersample_z=undersample_z,max_arxiv_size=max_arxiv_size,
                               verbose=False,paramList=paramList,train=True,emu_type=emu_type)
elif emu_type=='linear':
    # specify parameters to emulate
    emulate_slope=False
    emulate_growth=False
    emulate_running=False
    emulate_pressure=True
    emu=linear_emulator.LinearEmulator(basedir,p1d_label,skewers_label,kmax_Mpc=kmax_Mpc,
                                undersample_z=undersample_z,max_arxiv_size=max_arxiv_size,
                                emulate_slope=emulate_slope,emulate_running=emulate_running,
                                emulate_pressure=emulate_pressure,emulate_growth=emulate_growth,verbose=False)
elif emu_type=='NGP':
    emu=simplest_emulator.SimplestEmulator(basedir,p1d_label,skewers_label,
                                undersample_z=undersample_z,max_arxiv_size=max_arxiv_size,
                                verbose=False)
elif emu_type=='mf':
    emu=mf_emulator.MeanFluxEmulator(basedir,p1d_label,skewers_label,
                                kmax_Mpc=kmax_Mpc,emu_type='k_bin',
                                max_arxiv_size=None,
                                verbose=True,paramList=paramList,train=True)
else:
    raise ValueError('wrong emulator type '+emu_type)

0 0.05 0.0 <mf< 0.15000000000000002
1 0.15 0.04999999999999999 <mf< 0.25
2 0.25 0.15 <mf< 0.35
3 0.35 0.24999999999999997 <mf< 0.44999999999999996
4 0.44999999999999996 0.35 <mf< 0.5499999999999999
5 0.5499999999999999 0.44999999999999996 <mf< 0.6499999999999999
6 0.65 0.55 <mf< 0.75
7 0.75 0.65 <mf< 0.85
8 0.85 0.75 <mf< 0.95
9 0.95 0.85 <mf< 1.0
latin hyper-cube data {'param_space': {'Delta2_star': {'ip': 0, 'min_val': 0.25, 'max_val': 0.45, 'z_star': 3.0, 'kp_Mpc': 0.7, 'latex': '$\\Delta^2_\\star$'}, 'n_star': {'ip': 1, 'min_val': -2.35, 'max_val': -2.25, 'z_star': 3.0, 'kp_Mpc': 0.7, 'latex': '$n_\\star$'}, 'heat_amp': {'ip': 2, 'min_val': 0.3, 'max_val': 3.0, 'latex': '$H_A$'}, 'heat_slo': {'ip': 3, 'min_val': -1.5, 'max_val': 1.5, 'latex': '$H_S$'}, 'z_rei': {'ip': 4, 'min_val': 5.5, 'max_val': 16.0, 'latex': '$z_r$'}}, 'nsamples': 50, 'samples': {'0': [0.392, -2.319, 2.757, -0.4500000000000002, 11.905000000000001], '1': [0.268, -2.327, 1.0830000000000002, -1.41, 6.025], '2': [0

37 sample has sim params = [0.316, -2.261, 0.597, 1.17, 14.424999999999999]
simulation has 3 redshifts
undersample_z = 1
38 sample has sim params = [0.252, -2.321, 0.5429999999999999, -0.1499999999999999, 14.845]
simulation has 3 redshifts
undersample_z = 1
39 sample has sim params = [0.40800000000000003, -2.273, 1.7850000000000004, -0.33000000000000007, 10.434999999999999]
simulation has 3 redshifts
undersample_z = 1
40 sample has sim params = [0.42000000000000004, -2.317, 1.461, 0.6299999999999999, 7.915]
simulation has 3 redshifts
undersample_z = 1
41 sample has sim params = [0.336, -2.339, 2.4330000000000003, 0.9300000000000002, 8.965]
simulation has 3 redshifts
undersample_z = 1
42 sample has sim params = [0.348, -2.275, 1.569, 1.4699999999999998, 11.065000000000001]
simulation has 3 redshifts
undersample_z = 1
43 sample has sim params = [0.444, -2.309, 1.299, -1.0499999999999998, 13.795]
simulation has 3 redshifts
undersample_z = 1
44 sample has sim params = [0.43200000000000005,

 /home/font/anaconda3/lib/python3.6/site-packages/paramz/parameterized.py:61: RuntimeWarning:Don't forget to initialize by self.initialize_parameter()!


6750 initial entries
use 353/6750 entries
build emulator 1/10, <F>=0.150
Loading emulator using a specific arxiv, not the one set in basedir
Rescaled params to unity volume
Cannot load emulators with non-standard training data
will train GP emulator
Training GP on 353 points
Optimised
6750 initial entries
use 859/6750 entries
build emulator 2/10, <F>=0.250
Loading emulator using a specific arxiv, not the one set in basedir
Rescaled params to unity volume
Cannot load emulators with non-standard training data
will train GP emulator
Training GP on 859 points
Optimised
6750 initial entries
use 1221/6750 entries
build emulator 3/10, <F>=0.350
Loading emulator using a specific arxiv, not the one set in basedir
Rescaled params to unity volume
Cannot load emulators with non-standard training data
will train GP emulator
Training GP on 1221 points
Optimised
6750 initial entries
use 1409/6750 entries
build emulator 4/10, <F>=0.450
Loading emulator using a specific arxiv, not the one set in basedi

In [4]:
#theory=lya_theory.LyaTheory(zs,emulator=emu)
theory=lya_theory.get_mock_theory(zs,emulator=emu)

Note: redshifts have been re-sorted (earliest first)


In [9]:
# specify free parameters in likelihood
#free_parameters=['ln_kF_0']
#free_parameters=['ln_tau_0','ln_tau_1']
free_parameters=['ln_tau_0','ln_tau_1','ln_gamma_0','ln_kF_0','T0_1','T0_2','T0_3']
min_kp_kms=0.003
# ignore emulator covariance for now
ignore_emu_cov=True
like=likelihood.Likelihood(data=data,theory=theory,free_parameters=free_parameters,min_kp_kms=min_kp_kms,
                           verbose=False,ignore_emu_cov=ignore_emu_cov)

In [10]:
sampler = emcee_sampler.EmceeSampler(like=like,verbose=True)

use input likelihood
setup with 70 walkers
set 70 walkers with 7 dimensions
done setting up sampler


In [11]:
for p in sampler.like.free_params:
    print(p.name,p.value,p.min_value,p.max_value)

ln_tau_0 -0.7945801724139945 -1.5 -0.5
ln_tau_1 3.18 2.5 3.8
T0_1 0.44 0 1
T0_2 9.37 7 11
T0_3 -1.33 -2 -0.5
ln_gamma_0 0.3364722366212129 0.09531017980432493 0.6931471805599453
ln_kF_0 -1.7719568419318752 -2.995732273553991 -0.6931471805599453


### Run burn-in chains

In [12]:
sampler.like.go_silent()

In [13]:
sampler.run_burn_in(nsteps=500)

start burn-in, will do 500 steps
0 [0.52023417 0.49215279 0.53294901 0.49405646 0.51485171 0.49047791
 0.51025356]
20 [0.54513604 0.53062789 0.5351577  0.52735841 0.50670552 0.43345678
 0.50571021]
40 [0.53680841 0.54122643 0.55314668 0.55306843 0.52904248 0.40412821
 0.49971177]
60 [0.53931608 0.54021831 0.58011567 0.57603115 0.55039811 0.35963758
 0.48407426]
80 [0.53849579 0.53712541 0.57795931 0.58802607 0.54938313 0.32958824
 0.47416674]
100 [0.54389385 0.52799845 0.58376926 0.599287   0.57357202 0.29589983
 0.45720759]
120 [0.54494768 0.51378941 0.57233925 0.60789028 0.58419948 0.27635126
 0.43745021]
140 [0.54641564 0.50932805 0.57089329 0.60658996 0.57556998 0.27342946
 0.43157888]
160 [0.54909705 0.5157382  0.5859034  0.60341151 0.56018145 0.26443181
 0.42635477]
180 [0.54773507 0.50681722 0.57604298 0.61441826 0.56756827 0.26870719
 0.42272681]
200 [0.55191171 0.50811118 0.57461514 0.61558285 0.56340642 0.27034002
 0.41338031]
220 [0.54910192 0.51140907 0.56554155 0.61908671 

### Run main chains

In [ ]:
sampler.run_chains(nsteps=2000)

In [ ]:
print("Mean acceptance fraction: {0:.3f}".format(np.mean(sampler.sampler.acceptance_fraction)))

### Plot results in cube units (0 < x < 1)

In [ ]:
sampler.plot_histograms(cube=True)

In [ ]:
sampler.plot_corner(cube=True)

### Plot results for dimensional parameters

In [ ]:
sampler.plot_histograms(cube=False)

In [ ]:
sampler.plot_corner(cube=False)

### Use likelihood object to make other useful plots

In [ ]:
# get best fit model
ind = np.unravel_index(np.argmax(sampler.sampler.lnprobability, axis=None), sampler.sampler.lnprobability.shape)
best_fit=sampler.sampler.chain[ind]

In [ ]:
sampler.like.plot_p1d(values=best_fit,plot_every_iz=1)

In [ ]:
sampler.like.overplot_emulator_calls('gamma','sigT_Mpc',values=best_fit)

In [ ]:
sampler.like.overplot_emulator_calls('mF','gamma',values=best_fit)

In [ ]:
sampler.like.overplot_emulator_calls('Delta2_p','mF',values=best_fit)

In [ ]:
sampler.like.overplot_emulator_calls('alpha_p','n_p',values=best_fit)

In [ ]:
sampler.like.overplot_emulator_calls('sigT_Mpc','mF',values=best_fit)

In [ ]:
sampler.like.overplot_emulator_calls('mF','sigT_Mpc',values=best_fit)

In [ ]:
sampler.like.overplot_emulator_calls('kF_Mpc','sigT_Mpc',values=best_fit)

In [ ]:
sampler.like.overplot_emulator_calls('kF_Mpc','mF',values=best_fit)

### Write chain to file for later use

In [ ]:
chain_filename='test'
sampler.write_chain_to_file(chain_filename)

### Read pre-computed chain

In [ ]:
new_sampler = emcee_sampler.EmceeSampler(emulator=emu,free_parameters=free_parameters,
                                         read_chain_file=chain_filename,verbose=True)

In [ ]:
new_sampler.plot_corner(cube=False)

In [ ]:
new_sampler.plot_corner(cube=True)

In [ ]:
new_sampler.plot_histograms(cube=False)